In [1]:
from influx_interact import *
from clean_data import *
import pandas as pd


# Example of InfluxDB download.

In [2]:
#Define a few variables with the name of your bucket, organization, and token.
bucket = "SKYSPARK"
org = "UBC"
#UDL provides public users READ access to the InfluxDB 2.0 instance via this token
token = "insert public token here"
url="http://206.12.92.81:8086"

In [3]:
test = influx_class(org, url, bucket, token)
id = ["r:p:ubcv:r:205b0392-31f31280 Campus Energy Centre Campus HW Main Meter Power", "r:p:ubcv:r:205b1697-84986d73 Campus Energy Centre Campus HW Main Meter Leaving Water Temperature", "r:p:ubcv:r:205b0343-70d7c00c Campus Energy Centre Campus HW Main Meter Energy", "r:p:ubcv:r:205b03d6-b9859d31 Campus Energy Centre Campus HW Main Meter Flow"]

In [4]:
testdf = test.make_query("Campus Energy Centre", id)

In [5]:
# split sensor
main_bucket = split_sensors(testdf)

In [6]:
main_bucket[0].iloc[0:5,:]

,DateTime,Value,navName,ID,unit
0,2020-06-23 01:30:00+00:00,0.50000,Energy,r:p:ubcv:r:205b0343-70d7c00c Campus Energy Cen...,MWh
1,2020-06-23 01:45:00+00:00,0.50000,Energy,r:p:ubcv:r:205b0343-70d7c00c Campus Energy Cen...,MWh
2,2020-06-23 02:00:00+00:00,0.59375,Energy,r:p:ubcv:r:205b0343-70d7c00c Campus Energy Cen...,MWh
3,2020-06-23 02:15:00+00:00,0.40625,Energy,r:p:ubcv:r:205b0343-70d7c00c Campus Energy Cen...,MWh
4,2020-06-23 02:30:00+00:00,0.59375,Energy,r:p:ubcv:r:205b0343-70d7c00c Campus Energy Cen...,MWh


# Usng CSV Data

Since we don't have trained data on the above dataframe we will be using a modified csv file for example.

### Setting up Training Data
This would be done through influx_interact

In [8]:
training_data = pd.read_csv('../../data/testing-data/trained_bucket.csv')

training_data = training_data.rename(columns={'Anomaly':'AH'})
# Setup our training data to only have human labelled anomalies 
# and no machine labelled ones
# Remember True and False on 'AH' are verified true or false
training_data['AM'] = False

### Setting up Main Bucket
This would be done through influx_interact

In [9]:
main_bucket[0] = pd.read_csv('../../data/testing-data/main_bucket.csv')



### Standardize data

In [10]:
for i in range(0, len(main_bucket)):
    main_bucket[i]['Stand_Val'] = standardize_values(main_bucket[i][['Value']])

### Combining Training and Main Bucket

This would usually be added to the for loop for every sensor but seeing how we only have one set of training data in the example.

In [11]:
joined_df = add_anomalies(main_bucket[0],training_data)


### Group Check

This would look if the dataframe sensor belongs to a group and return the name of the group. Returns None if the sensor does not.  This can be modified for however we plan to set up model groups.  Need to talk to Mitch about this.

In [12]:
group_check(joined_df)

'Group A'

Looking at the dataframe produced.  Just need to chat with Mitch about what he needs coming into his detection model.

In [13]:
joined_df.iloc[0:5,:]

,AH,AM,Datetime,ID,Stand_Val,Value
0,NaN,False,2020-01-01 07:58:00,Campus Energy Centre Campus HW Main Meter Power,-0.487452,9.6
1,NaN,False,2020-01-01 08:00:00,Campus Energy Centre Campus HW Main Meter Power,-0.487452,9.6
2,NaN,False,2020-01-01 08:01:00,Campus Energy Centre Campus HW Main Meter Power,-0.406726,10.3
3,NaN,False,2020-01-01 08:04:00,Campus Energy Centre Campus HW Main Meter Power,-0.326000,11.0
4,NaN,False,2020-01-01 08:07:00,Campus Energy Centre Campus HW Main Meter Power,-0.406726,10.3
